# Carnival Demo

In [ ]:
import serial
from LatLongMap import latitude_values, longitude_values
from gpt import GPTWaterLevelModel
import torch

In [ ]:
device = 'cpu'

In [ ]:
def getIndexForXposition(x, start_value = 44.25, end_value = 8.00, step = 0.01, num_indices = 275):
    normalised_position = (start_value - x)/(start_value - end_value)
    index = normalised_position * (num_indices - 1)
    index = abs(int(index))
    return index

In [ ]:
def createDataForModel(tensor, stack_vector):
    # If stack_vector is initially of size [1, 0, 0], initialize it with the new tensors
    if stack_vector.size() == torch.Size([1, 0, 0]):
        stack_vector = torch.stack(tensor).unsqueeze(0).unsqueeze(0)
    else:
        # Otherwise, concatenate the new tensors along the second dimension
        new_tensor = torch.stack(tensor).unsqueeze(0).unsqueeze(0)
        stack_vector = torch.cat((stack_vector, new_tensor), dim=1)    
    return stack_vector

In [ ]:
def getNextCoordinates(stack_vector, count=4):
    nextLatLong = torch.empty((count, 0))
    #Get the lat and long from the last element of the vector
    last_entry_lat_long = stack_vector[0, -1, 1:]
    #Find the index of current latitude
    index = torch.where(latitude_values == last_entry_lat_long[0])[0]
    for i in range(count):
        index = index+1
        tensor = torch.tensor(latitude_values[index], longitude_values[index], dtype='float16')
        if nextLatLong is torch.empty((count,0)):
            nextLatLong = torch.stack(tensor)
        else:
            nextLatLong = torch.cat((nextLatLong, tensor),dim = 0)
    return nextLatLong   

In [ ]:
def getWaterLevelPredictions(CurrentWaterLevels, model):
    next_coordinates = getNextCoordinates(CurrentWaterLevels)
    predicted_water_levels = model.generate(CurrentWaterLevels, 4, next_coordinates) 
    print(f'Current 8(upto) water levels = {[round(value, 4) for value in CurrentWaterLevels[:,:,-1].tolist()[0]]}' )
    print(f'Predicted next 4 water levels = {[round(value,4) for value in predicted_water_levels[:,8:,-1].tolist()[0]]}' )
    pass

In [ ]:
def read_data(ser):    
    if ser.in_waiting > 0:
        raw_data = ser.readline().strip()
        raw_data.decode('latin1')
        #buf = ser.readline().decode('utf-8').rstrip()
        buf = str(raw_data).rstrip("\n")
        #print(buf)
        start = buf.find('[')
        #print(start)
        buf = buf[start+1:]
        end = buf.find(']')
        #print(end)
        msg_parts = buf[:end].split(";") # buf now has depth, x-axis
        curWaterDepth = msg_parts[0]
        index = getIndexForXposition(msg_parts[1])
        lat = latitude_values[index]
        long = longitude_values[index]
        tensor = torch.tensor([curWaterDepth, lat, long], dtype=torch.float32, device=device)      
        return tensor



In [ ]:
len(latitude_values), len(longitude_values)

In [ ]:
ser.close() 

In [ ]:
running = True
ser = serial.Serial("COM7", 9600)
saved_model = GPTWaterLevelModel(800).to(device=device)
saved_model.load_state_dict(torch.load('saved_model.pth'))
LatLongDepthTensor = torch.empty((1, 0, 0))
while running:
    tensor = read_data(ser)
    LatLongDepthTensor = createDataForModel(tensor, LatLongDepthTensor)
    getWaterLevelPredictions(LatLongDepthTensor, saved_model)

In [ ]:
ser.close() 

In [ ]:
print(serial.__version__)

In [ ]:
print(ser.in_waiting)